In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [3]:
prop = pickle.load(open("../../Data.p","rb"))

In [4]:
prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72474 entries, 0 to 72473
Data columns (total 35 columns):
id                            72474 non-null object
created_on                    72474 non-null object
operation                     72474 non-null object
property_type                 72474 non-null object
place_name                    72446 non-null object
place_with_parent_names       72474 non-null object
country_name                  72474 non-null object
state_name                    72474 non-null object
geonames_id                   62030 non-null float64
lat-lon                       48174 non-null object
lat                           48174 non-null float64
lon                           48174 non-null float64
price                         72474 non-null float64
currency                      72473 non-null object
price_aprox_local_currency    72474 non-null float64
price_aprox_usd               72474 non-null float64
surface_total_in_m2           72474 non-null float64


In [5]:
#OJO CON ESTO
proper = pickle.load(open("../../test_data_full.p", "rb"))
#agrego el set de pruebas con los datos posta al de entrenamiento
propTOT = pd.merge(proper, prop, how="outer")
#completo las superficies totales con las cubiertas
propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[12] if(np.all(pd.isnull(row[11]))) else row[11], axis=1)
#cortamos por precios
propTOT = propTOT[(propTOT['price_usd_per_m2']<7000)]

## Division del set de Entrenamiento

In [6]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd', 'state_name']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd', 'state_name']]

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

### Se le asigna un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 3
    if p == 'house':
        return 2.25
    if p == 'store':
        return 6.5
    return 2.5

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor numerico a la ubicacion CABA o GBA

In [9]:
def asignarLT( p ):
    if p == 'Capital Federal':
        return 0
    return 10

In [10]:
p['location_type'] = p.apply(lambda x: asignarLT(x[10]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [11]:
def asignarPV(name, dic):
    return dic[name]   

In [12]:
hoodGroups = p[["place_name", "price_usd_per_m2", "Cant_ParadasTransporte"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [13]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [14]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [15]:
pEnt = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd', 'Cant_ColeYUniv', 'location_type']]

In [16]:
pEnt.dropna(inplace=True)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### estimo las superficies para el set de prueba

In [17]:
hoodGroups = p[["place_name", 'surface_total_in_m2']]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("surface_total_in_m2", "mean"), ascending=False)

In [18]:
HGSdics = {}
for row in hoodGroups.iterrows():
    name = row[1][0]
    supr = row[1][1]
    HGSdics[name] = supr

# Se carga el set de Prueba

# PRUEBA Random Forest Regresivo

In [19]:
proper = pickle.load(open("../../test_data_full.p", "rb"))

#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd', 'state_name']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[4]))) else row[4], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[4]))) else row[4], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd', 'state_name']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

def asignarLT( p ):
    if p == 'Capital Federal':
        return 0
    return 1

p['location_type'] = p.apply(lambda x: asignarLT(x[10]), axis = 1)

#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd', 'place_name']]

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Estimo los valores nan del set de pruebas de superficies

In [20]:
def asignarSupFaltantes(HGSdics, sup, barrio):
    if (np.isnan(sup) & HGSdics.has_key(barrio)):
        return HGSdics[barrio]
    return sup

In [21]:
pPru['surface_total_in_m2'] = pPru.apply(lambda x: asignarSupFaltantes(HGSdics, x[1], x[9]), axis=1)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Prediccion

In [22]:
#pEnt['surface_total_in_m2_n'] = pEnt.apply(lambda x: x[1]*10, axis=1)
#pPru['surface_total_in_m2_n'] = pEnt.apply(lambda x: x[1]*10, axis=1)

In [23]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'place_value']],pEnt[['price_aprox_usd']],test_size=0.0)

In [24]:
#definir el modelo a probar
model = GradientBoostingRegressor(loss = 'ls', n_estimators = 500, max_depth = 5, learning_rate = 0.15)

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_aprox_usd'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.15, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [26]:
#prediccion 
pred = mean_squared_error(Ytrn['price_aprox_usd'], model.predict(Xtrn))
print ('RMSE_Price', pred) 

('RMSE_Price', 14589498561.084095)


In [27]:
#borrar esto, es solo para probar
#pPru.dropna(subset=['price_aprox_usd'], inplace=True)

In [28]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value']])

In [29]:
#prediccion 
#pred = mean_squared_error(pPru['price_aprox_usd'], model.predict(pPru_imp))
#print ('RMSE_Price', pred)

In [30]:
pPru['price_usd'] = model.predict(pPru_imp)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
result = pd.DataFrame()

In [32]:
result["id"] = proper["id"]

In [33]:
result["price_usd"] = pPru['price_usd']

In [34]:
result.head()

,id,price_usd
0,2316505,156698.647417
1,2320880,226138.610824
2,2321742,198465.297329
3,2276796,116021.544982
4,2330364,100283.408853


In [35]:
result.to_csv(index=False, path_or_buf="resultsGB2PhiAlpha.csv")

In [36]:
#para ver mas o menos que tan lejos quedo
result["real_price_usd"] = proper['price_aprox_usd']

In [37]:
result.head()

,id,price_usd,real_price_usd
0,2316505,156698.647417,11111.00
1,2320880,226138.610824,11111.00
2,2321742,198465.297329,12554.74
3,2276796,116021.544982,13000.00
4,2330364,100283.408853,13000.00
